# EEG SSVEP Test

## 一、Dataset类创建

In [1]:
from typing import Any, Callable, Dict, List, Optional, Tuple
from torchvision.datasets import MNIST
from torchvision import transforms
from torch.utils.data import DataLoader, Dataset
import torch
import torch.nn as nn
import numpy as np
import scipy

class Benchmark(Dataset):
    
    classes = {
        
    }
    
    stim_event_freq = [8., 8.2, 8.4, 8.6, 8.8, 9., 9.2, 9.4, 9.6, 9.8, 10., 10.2, 10.4, 10.6,
                       10.8, 11., 11.2, 11.4, 11.6, 11.8, 12., 12.2, 12.4, 12.6, 12.8, 13., 13.2, 13.4,
                       13.6, 13.8, 14., 14.2, 14.4, 14.6, 14.8, 15., 15.2, 15.4, 15.6, 15.8]
    
    def __init__(
        self,
        root: str = '',
        train: bool = True,
        transform: Optional[Callable] = None,
        target_transform: Optional[Callable] = None,
    ) -> None:
        super(Dataset).__init__()
        self.root = root
        self.train = train
        self.transform = transform
        self.target_transform = target_transform
        self.data, self.pre_data, self.label = self.load_data()
    
    def load_data(self) -> Tuple[np.ndarray, np.ndarray, np.ndarray]:
        channels = [53, 54, 55, 57, 58, 59, 61, 62, 63]
        channels = [i - 1 for i in channels]
        
        if self.train:
            # train data
            data = np.zeros((200*35, len(channels), 1375))
            pre_data = np.zeros((200*35, len(channels), 125))
            label = np.zeros(200*35, dtype=int)
        else:
            # test data
            data = np.zeros((40*35, len(channels), 1375))
            pre_data = np.zeros((40*35, len(channels), 125))
            label = np.zeros(40*35, dtype=int)
            
        for sub_num in range(1, 36):
            f = scipy.io.loadmat(self.root + f"/S{sub_num}.mat")
            print(f"mat{sub_num}文件大小: {f['data'].shape}")
            for block in range(6):
                for target in range(40):
                    if self.train and block!=5:
                        data[(sub_num - 1) * 200 + block * 40 + target] = f["data"][channels, 125:, target, block]
                        pre_data[(sub_num - 1) * 200 + block * 40 + target] = f["data"][channels, :125, target, block]
                        label[(sub_num - 1) * 200 + block * 40 + target] = int(target + 1)
                    elif not self.train and block==5:
                        data[(sub_num - 1) * 40 + target] = f["data"][channels, 125:, target, block]
                        pre_data[(sub_num - 1) * 40 + target] = f["data"][channels, :125, target, block]
                        label[(sub_num - 1) * 40 + target] = int(target + 1)
        return data, pre_data, label
    
    def __len__(self) -> int:
        return len(self.data)
    
    def __getitem__(self, index) -> Tuple[Any, Any]:
        eeg, target = self.data[index], self.label[index]
        
        if self.transform is not None:
            eeg = self.transform(eeg)

        if self.target_transform is not None:
            target = self.target_transform(target)
        
        return eeg, target


In [2]:
train_data = Benchmark("D:\Datasets\BCI\SSVEP Benchmark", train = True, 
                       transform = transforms.Compose([
                           transforms.ToTensor(),
                       ]))
test_data = Benchmark("D:\Datasets\BCI\SSVEP Benchmark", train = False, 
                       transform = transforms.Compose([
                           transforms.ToTensor(),
                       ]))

mat1文件大小: (64, 1500, 40, 6)
mat2文件大小: (64, 1500, 40, 6)
mat3文件大小: (64, 1500, 40, 6)
mat4文件大小: (64, 1500, 40, 6)
mat5文件大小: (64, 1500, 40, 6)
mat6文件大小: (64, 1500, 40, 6)
mat7文件大小: (64, 1500, 40, 6)
mat8文件大小: (64, 1500, 40, 6)
mat9文件大小: (64, 1500, 40, 6)
mat10文件大小: (64, 1500, 40, 6)
mat11文件大小: (64, 1500, 40, 6)
mat12文件大小: (64, 1500, 40, 6)
mat13文件大小: (64, 1500, 40, 6)
mat14文件大小: (64, 1500, 40, 6)
mat15文件大小: (64, 1500, 40, 6)
mat16文件大小: (64, 1500, 40, 6)
mat17文件大小: (64, 1500, 40, 6)
mat18文件大小: (64, 1500, 40, 6)
mat19文件大小: (64, 1500, 40, 6)
mat20文件大小: (64, 1500, 40, 6)
mat21文件大小: (64, 1500, 40, 6)
mat22文件大小: (64, 1500, 40, 6)
mat23文件大小: (64, 1500, 40, 6)
mat24文件大小: (64, 1500, 40, 6)
mat25文件大小: (64, 1500, 40, 6)
mat26文件大小: (64, 1500, 40, 6)
mat27文件大小: (64, 1500, 40, 6)
mat28文件大小: (64, 1500, 40, 6)
mat29文件大小: (64, 1500, 40, 6)
mat30文件大小: (64, 1500, 40, 6)
mat31文件大小: (64, 1500, 40, 6)
mat32文件大小: (64, 1500, 40, 6)
mat33文件大小: (64, 1500, 40, 6)
mat34文件大小: (64, 1500, 40, 6)
mat35文件大小: (64, 1500, 4

In [3]:
print("train_data:\n", len(train_data))
print(f"shape: {train_data[0][0].shape}, type: {type(train_data[0][0])}")

print("test_data:\n", len(test_data))
print(f"shape: {test_data[0][0].shape}, type: {type(test_data[0][0])}")

train_data:
 7000
shape: torch.Size([1, 9, 1375]), type: <class 'torch.Tensor'>
test_data:
 1400
shape: torch.Size([1, 9, 1375]), type: <class 'torch.Tensor'>


## 二、EEGNet构建

In [55]:
from torchsummary import summary
class DepthwiseSeparableConv2d(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size, depth_multiplier=1):
        super(DepthwiseSeparableConv2d, self).__init__()
        self.depthwise = nn.Conv2d(in_channels, in_channels * depth_multiplier, kernel_size=kernel_size,
                                   stride=(1, 1), padding=(0, 0 if kernel_size[0]>kernel_size[1] else max(kernel_size)//2), groups=in_channels, bias=False)
        self.pointwise = nn.Conv2d(in_channels * depth_multiplier, out_channels, kernel_size=(1, 1),
                                   stride=(1, 1), padding=(0, 0), bias=False)

    def forward(self, x):
        x = self.depthwise(x)
        x = self.pointwise(x)
        return x


class EEGNet(nn.Module):
    def __init__(self, nb_classes, Chans=64, Samples=128, dropoutRate=0.5, kernLength=64,
                 F1=8, D=2, F2=16, norm_rate=0.25, dropoutType='Dropout'):
        super(EEGNet, self).__init__()

        if dropoutType == 'SpatialDropout2D':
            self.dropoutType = nn.Dropout2d
        elif dropoutType == 'Dropout':
            self.dropoutType = nn.Dropout
        else:
            raise ValueError('dropoutType must be one of SpatialDropout2D '
                             'or Dropout, passed as a string.')

        self.block1 = nn.Sequential(
            nn.Conv2d(1, F1, (1, kernLength), padding=(0, kernLength//2), bias=False),
            nn.BatchNorm2d(F1),
            DepthwiseSeparableConv2d(F1, F1, kernel_size=(Chans, 1), depth_multiplier=D),
            nn.BatchNorm2d(F1),
            nn.ELU(),
            nn.AvgPool2d((1, 4)),
            self.dropoutType(dropoutRate)
        )

        self.block2 = nn.Sequential(
            DepthwiseSeparableConv2d(F1, F2, kernel_size=(1, 16), depth_multiplier=1),
            nn.BatchNorm2d(F2),
            nn.ELU(),
            nn.AvgPool2d((1, 8)),
            self.dropoutType(dropoutRate)
        )

        self.block3 = nn.Sequential(
            nn.Flatten(),
            nn.Linear(F2*int(np.floor((np.floor((Samples+1)/4)+1)/8)), nb_classes),
            nn.Softmax(dim=1)
        )

    def forward(self, input):
        x = self.block1(input)
        x = self.block2(x)
        x = self.block3(x)
        return x

nb_classes = 40
Chans = 9
Samples = 1375
model = EEGNet(nb_classes, Chans, Samples)
print(model)
print(summary(model, input_size=(1, 9, 1375)))

EEGNet(
  (block1): Sequential(
    (0): Conv2d(1, 8, kernel_size=(1, 64), stride=(1, 1), padding=(0, 32), bias=False)
    (1): BatchNorm2d(8, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): DepthwiseSeparableConv2d(
      (depthwise): Conv2d(8, 16, kernel_size=(9, 1), stride=(1, 1), groups=8, bias=False)
      (pointwise): Conv2d(16, 8, kernel_size=(1, 1), stride=(1, 1), bias=False)
    )
    (3): BatchNorm2d(8, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (4): ELU(alpha=1.0)
    (5): AvgPool2d(kernel_size=(1, 4), stride=(1, 4), padding=0)
    (6): Dropout(p=0.5, inplace=False)
  )
  (block2): Sequential(
    (0): DepthwiseSeparableConv2d(
      (depthwise): Conv2d(8, 8, kernel_size=(1, 16), stride=(1, 1), padding=(0, 8), groups=8, bias=False)
      (pointwise): Conv2d(8, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
    )
    (1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ELU(alpha=1.0

In [52]:
class Light(nn.Module):
    def __init__(self):
        super(Light, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=6, kernel_size=(1, 4), stride=1, padding=(0, 2))  # 3*3卷积核， （1， 1）步长

    def forward(self, x):
        x = self.conv1(x)
        return x
light = Light()
pic = torch.rand((3, 9, 35))
print(light(pic).shape)

torch.Size([6, 9, 36])


In [30]:
dataloader = DataLoader(train_data, batch_size=16, shuffle=True)
for i, (eegs, labels) in enumerate(dataloader):
    print(f"{i}:", eegs.shape)

0: torch.Size([16, 1, 9, 1375])
1: torch.Size([16, 1, 9, 1375])
2: torch.Size([16, 1, 9, 1375])
3: torch.Size([16, 1, 9, 1375])
4: torch.Size([16, 1, 9, 1375])
5: torch.Size([16, 1, 9, 1375])
6: torch.Size([16, 1, 9, 1375])
7: torch.Size([16, 1, 9, 1375])
8: torch.Size([16, 1, 9, 1375])
9: torch.Size([16, 1, 9, 1375])
10: torch.Size([16, 1, 9, 1375])
11: torch.Size([16, 1, 9, 1375])
12: torch.Size([16, 1, 9, 1375])
13: torch.Size([16, 1, 9, 1375])
14: torch.Size([16, 1, 9, 1375])
15: torch.Size([16, 1, 9, 1375])
16: torch.Size([16, 1, 9, 1375])
17: torch.Size([16, 1, 9, 1375])
18: torch.Size([16, 1, 9, 1375])
19: torch.Size([16, 1, 9, 1375])
20: torch.Size([16, 1, 9, 1375])
21: torch.Size([16, 1, 9, 1375])
22: torch.Size([16, 1, 9, 1375])
23: torch.Size([16, 1, 9, 1375])
24: torch.Size([16, 1, 9, 1375])
25: torch.Size([16, 1, 9, 1375])
26: torch.Size([16, 1, 9, 1375])
27: torch.Size([16, 1, 9, 1375])
28: torch.Size([16, 1, 9, 1375])
29: torch.Size([16, 1, 9, 1375])
30: torch.Size([16, 

[ 8.   8.2  8.4  8.6  8.8  9.   9.2  9.4  9.6  9.8 10.  10.2 10.4 10.6
 10.8 11.  11.2 11.4 11.6 11.8 12.  12.2 12.4 12.6 12.8 13.  13.2 13.4
 13.6 13.8 14.  14.2 14.4 14.6 14.8 15.  15.2 15.4 15.6 15.8]
